In [1]:
import torch
import pandas as pd
import numpy as np
import syft as sy
import copy
hook = sy.TorchHook(torch)
from torch import nn
import torch.nn.functional as F
from torch import optim
from sklearn.model_selection import train_test_split

In [2]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

In [3]:
dataframe=np.genfromtxt("diabetes.csv",delimiter=',',skip_header=1)

In [4]:
X=dataframe[:,0:8]
Y=dataframe[:,8]
Y=Y.reshape(768,1)
dtype=torch.float
device=torch.device("cpu")

In [5]:
from sklearn.preprocessing import StandardScaler
model=StandardScaler()
X=model.fit_transform(X)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [7]:
data = torch.from_numpy(X_train)
target = torch.from_numpy(y_train)
data,target=data.type(torch.FloatTensor),target.type(torch.FloatTensor)

In [8]:
data_length,data_width=data.shape
partition=0.01 #Input only from 0 to 0.5 for bobs data. Alice is 1- partition

In [9]:
bobs_data = data[0:int(data_length*partition)].send(bob)
bobs_target = target[0:int(data_length*partition)].send(bob)

alices_data = data[int(data_length*partition):].send(alice)
alices_target = target[int(data_length*partition):].send(alice)


In [10]:
# Iniitalize A Toy Model
model = nn.Linear(8,1)
loss= torch.nn.MSELoss()

In [11]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.001)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.001)

In [12]:
for i in range(1000):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = loss(bobs_pred,bobs_target)
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = loss(alices_pred,alices_target)
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    
#     print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

In [13]:
iterations = 10
worker_iters = 200

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.001)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.001)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model.forward(bobs_data)
        bobs_loss = loss(bobs_pred,bobs_target)
        bobs_loss.backward()        
        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model.forward(alices_data)
        alices_loss = loss(alices_pred,alices_target)
        alices_loss.backward()
        alices_opt.step()
#         alices_opt.step(alices_data.shape[0])
        alices_loss = alices_loss.get().data
#         print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    
    model.weight.data.set_(((alices_model.weight.data *(1- partition) + bobs_model.weight.data * partition) ).get())
    model.bias.data.set_(((alices_model.bias.data *(1- partition) + bobs_model.bias.data * partition) ).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.1601) Alice:tensor(0.3658)
Bob:tensor(0.0895) Alice:tensor(0.2566)
Bob:tensor(0.0599) Alice:tensor(0.2092)
Bob:tensor(0.0464) Alice:tensor(0.1868)
Bob:tensor(0.0399) Alice:tensor(0.1757)
Bob:tensor(0.0365) Alice:tensor(0.1700)
Bob:tensor(0.0346) Alice:tensor(0.1669)
Bob:tensor(0.0334) Alice:tensor(0.1652)
Bob:tensor(0.0326) Alice:tensor(0.1642)
Bob:tensor(0.0320) Alice:tensor(0.1637)


In [14]:
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)
X_test,y_test=X_test.type(torch.FloatTensor),y_test.type(torch.FloatTensor)
pred=model(X_test)

In [30]:
pred=torch.abs(torch.round((pred)))
output=pred-y_test

In [31]:
incorrect_labels=len(np.nonzero(output))

accuracy=1-(incorrect_labels/len(X_test))

accuracy

0.8311688311688312

In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_test.detach().numpy(),pred.detach().numpy()))

              precision    recall  f1-score   support

         0.0       0.85      0.92      0.88       107
         1.0       0.77      0.64      0.70        47

   micro avg       0.83      0.83      0.83       154
   macro avg       0.81      0.78      0.79       154
weighted avg       0.83      0.83      0.83       154

